## Requirements
```
pip install azure-storage-blob python-dotenv
```

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# use label csv, download data and store into local path
import os
from pathlib import Path
import sys
if ".." not in sys.path:
    sys.path.append("..")  # Append pieye module root to sys.path

from azure.storage.blob import BlobProperties, BlobServiceClient, ContainerClient
from dotenv import load_dotenv

from pieye.data import one_hot_encode_azureml_data_labels

load_dotenv()


STORAGE_NAME = "conveyorbeltstrg"
STORAGE_KEY = os.getenv("BLOB_STORAGE_KEY")
CONTAINER_NAME = "data"
DOWNLOAD_DIR = "../../data/images"

In [3]:
classes = ["empty", "bag", "damaged_bag", "stacked_bag", "others"]

## Download from Blob Storage

In [4]:
image_filepaths, labels = one_hot_encode_azureml_data_labels(
    "../../data/labeledDatapoints.csv",
    classes=classes,
    returns_full_path=True,
)

In [5]:
image_filepaths

0      unlabeled/ECAM001/2024091817/36/vid-2024091817...
1      unlabeled/ECAM001/2024091817/35/vid-2024091817...
2      unlabeled/ECAM001/2024091817/35/vid-2024091817...
3      unlabeled/ECAM001/2024091817/36/vid-2024091817...
4      unlabeled/ECAM001/2024091817/35/vid-2024091817...
                             ...                        
386    unlabeled/ECAM001/2024091817/36/vid-2024091817...
387    unlabeled/ECAM001/2024091817/37/vid-2024091817...
388    unlabeled/ECAM001/2024091817/38/vid-2024091817...
389    unlabeled/ECAM001/2024091817/36/vid-2024091817...
390    unlabeled/ECAM001/2024091817/39/vid-2024091817...
Name: Url, Length: 391, dtype: object

In [6]:
account_url = f"https://{STORAGE_NAME}.blob.core.windows.net"

with BlobServiceClient(account_url=account_url, credential=STORAGE_KEY) as blob_service_client:
    with blob_service_client.get_container_client(CONTAINER_NAME) as container_client:

        for filepath in image_filepaths:
            filename = filepath.split("/")[-1]
            with container_client.get_blob_client(filepath) as blob_client:
                download_file_path = Path(DOWNLOAD_DIR, filename)
                # Create local directories if they do not exist
                Path.mkdir(download_file_path.parent, parents=True, exist_ok=True)
                with open(download_file_path, "wb") as download_file:
                    try:
                        download_file.write(blob_client.download_blob().readall())
                    except Exception as e:
                        print(e)